# workflow agents scratch

In [ ]:
import os 
from langchain_openai import AzureChatOpenAI
import dotenv
from langgraph.graph import StateGraph, START, END 

dotenv.load_dotenv()


llm = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OpenAI_API_VERSION_3"),
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL_3'),
    model_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL_3'),
    api_key=os.getenv('OPENAI_API_KEY_3'),
    azure_endpoint=os.getenv('AZURE_OpenAI_ENDPOINT_3')
   )

llm.invoke('hello').content

'Hello! How can I assist you today? 😊'

In [ ]:
# schema for structured output 
from pydantic import BaseModel, Field
from typing import Optional 


class SearchQuery(BaseModel):
    search_query:Optional[str] = Field(None,description='Query that is optimized web search')
    justification: Optional[str] = Field(
        None, description='why this query is relevant to the users request'
    )

# Augment the LLM with schema for structured output 
structured_llm=llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
output=structured_llm.invoke('How does calcium CT score relate to high cholesterol?')
print(output.search_query)
print(output.justification)


calcium CT score high cholesterol relationship
Exploring the connection provides medical insights into heart health and risk assessment.
search_query='calcium CT score high cholesterol relationship' justification='Exploring the connection provides medical insights into heart health and risk assessment.'


In [9]:
# define the tool 
def multiply(a:int, b:int)-> int:
    return a* b 

# Augment the LLM with tools 
llm_with_tools=llm.bind_tools({multiply})

# Invoke the LLM with input that triggers the tool call 
msg=llm_with_tools.invoke('what is 10 times 3')

msg.tool_calls

[{'name': 'multiply',
  'args': {'a': 10, 'b': 3},
  'id': 'call_DDcyzYYt9XxLLfNm15SVeThA',
  'type': 'tool_call'}]